- modify fitness metric function
- add astype(int)
- modify bug
- add classification rules

# A DSL alongside a Genetic Algorithm applied to the ARC Dataset

In this notebook, we present a minimalistic *Domain Specific Language* for some of the ARC tasks.

We instroduce the language and how it can be used to precess the input in complex ways. We then implement an evaluation function able to run a such program against an input image. We also provide a program solution of a task as an exemple.

In a second time, we implement a simple genetic algorithm (based on a multiobjective and elitist strategy) that is able to generate programs written in this DSL and demonstrate its usage against the same ARC task previously solved by hand.

In [1]:
import numpy as np
import pandas as pd
import itertools
import random
import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'
training_tasks = sorted(os.listdir(training_path))
evaluation_tasks = sorted(os.listdir(evaluation_path))
test_tasks = sorted(os.listdir(test_path))
num2color = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown"]
color2num = {c: n for n, c in enumerate(num2color)}

In [2]:
# This code is used to display a task
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
def plot_one(ax, i,train_or_test,input_or_output):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    
    input_matrix = task[train_or_test][i][input_or_output]
    ax.imshow(input_matrix, cmap=cmap, norm=norm)
    ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
    ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
    ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(train_or_test + ' '+input_or_output)
    
def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(axs[0,i],i,'train','input')
        plot_one(axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(axs[0],0,'test','input')
        plot_one(axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(axs[0,i],i,'test','input')
            plot_one(axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 
    
# Display each output of the function
def show_image_list(images):
    """ Show each image contained in a list. """
    p = plt.figure().subplots(1, len(images))
    if len(images) > 1:
        for i, image in enumerate(images):
            p[i].imshow(image, cmap=cmap, norm=norm)
    elif len(images) == 1:
        p.imshow(images[0], cmap=cmap, norm=norm)

In [3]:
def create_df(folder_path):
    task_names_list = sorted(os.listdir(folder_path))
    task_list = []
    for task_name in task_names_list: 
        task_file = str(folder_path / task_name)
        with open(task_file, 'r') as f:
            task = json.load(f)
            task_list.append(task)
    
    df = pd.DataFrame()
    df['task_name'] = task_names_list
    df['task'] = task_list
    df['number_of_train_pairs'] = df['task'].apply(lambda x: len(x['train']))
    df['number_of_test_pairs'] = df['task'].apply(lambda x: len(x['test']))
    
    # Compare image sizes
    df['inputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['input']) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['input'][0]) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_shape'] = df['inputs_all_have_same_height'] * df['inputs_all_have_same_width']
    df['input_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'])
                     if (len(set([len(example['input']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['input_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) for example in task['train']])) == 1)
                     else np.nan
    )
    df['outputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['output']) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['output'][0]) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_shape'] = df['outputs_all_have_same_height'] * df['outputs_all_have_same_width']
    df['output_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'])
                     if (len(set([len(example['output']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['output_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0])
                     if (len(set([len(example['output'][0]) for example in task['train']])) == 1)
                     else np.nan
    )  
    df['in_each_pair_shape_doesnt_change'] = df['task'].apply(
        lambda task: np.prod([int(len(example['input'][0])==len(example['output'][0])
                                  and len(example['input'])==len(example['output'])
                                 ) for example in task['train']
                            ])
    )
    df['in_each_pair_shape_ratio_is_the_same'] = df['task'].apply(
        lambda task: (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1) * (
                      len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
    )
    df['o/i_height_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output']) / len(task['train'][0]['input'])
                     if (len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    df['o/i_width_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0]) / len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    
    # my idea ---------
    df["same_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input'])))== sum(sum(np.array(example['output'])))) for example in task['train']]))
    
    df["same_color_sum_in_edge"] = df['task'].apply(lambda task: 
                        np.all([int(sum(np.array(example['input'])[0,:]) +sum(np.array(example['input'])[:,0]) + 
                                    sum(np.array(example['input'])[-1,:]) +sum(np.array(example['input'])[:,-1])
                                    == 
                                    sum(np.array(example['output'])[0,:]) +sum(np.array(example['output'])[:,0]) + 
                                    sum(np.array(example['output'])[-1,:]) +sum(np.array(example['output'])[:,-1])) for example in task['train']]))
    
    df["io_color_kind_diff"] = df['task'].apply(lambda task: [len(np.unique(np.array(example['input']))) - len(np.unique(np.array(example['output']))) for example in task['train']])
    df["io_color_kind_diff_constant"] = df['io_color_kind_diff'].apply(lambda task: np.unique(np.array(task))[0] if len(np.unique(np.array(task)))==1 else -1)
    df["output_not_include_0"] = df['task'].apply(lambda task: np.all([np.all(np.array(example['output']) > 0) for example in task['train']]))
    df["increase_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) < sum(sum(np.array(example['output'])))) for example in task['train']]))
    df["decrease_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) > sum(sum(np.array(example['output'])))) for example in task['train']]))
    

    return df

training_descriptive_df = create_df(training_path)
evaluation_descriptive_df = create_df(evaluation_path)
test_descriptive_df = create_df(test_path)

In [4]:
def classification(row):
    # same shape and same color sum → xgboost
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.same_color_sum==1:
        return 1
    # same shape and increase color sum and include black in output　→ xgboost
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.increase_color_sum==1 and row.output_not_include_0 == 0:
        return 2
    # same shape and incrase color sum and no black in output
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.increase_color_sum==1 and row.output_not_include_0 == 1:
        return 3
    # same shape and decrease color sum → xgboost
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.decrease_color_sum==1:
        return 4
    # different shape and decrease color sum
    elif row["in_each_pair_shape_doesnt_change"] == 0 and row.decrease_color_sum==1:
        return 5
    # different shape and increase color sum
    elif row["in_each_pair_shape_doesnt_change"] == 0 and row.increase_color_sum==1:
        return 6
    # different shape and same color sum
    elif row["in_each_pair_shape_doesnt_change"] == 0 and row.same_color_sum==1:
        return 7
    # otherwise
    else:
        return 8
training_descriptive_df["class"] = training_descriptive_df.apply(lambda x: classification(x), axis=1)
evaluation_descriptive_df["class"] = evaluation_descriptive_df.apply(lambda x: classification(x), axis=1)
test_descriptive_df["class"] = test_descriptive_df.apply(lambda x: classification(x), axis=1)

# Domain Specific Language (DSL)

We will build a domain specific language specialized on processing list of images. To allow easy chaining of keyword from this language together, each *function* provided by this language will be take one or more images and transform it to none, one or more. The final result of our program will then be a list of images.

The DSL is so constituted by a collection of functions of type `np.array -> [np.array]` and `[np.array] -> [np.array]`.

The first kind of function take an image, and produce a list of images (for example, the image split by different colors). The second type of function take a list of images and produce a new list (for exemple, intersect).
[](http://)

## DSL Implementation

We start with the functions that take *one image* and produce an *a list of images*.](http://)

In [5]:
# np.array -> [np.array]
def groupByColor_unlifted(pixmap):
    """ Split an image into a collection of images with unique color """
    # Count the number of colors
    nb_colors = int(pixmap.max()) + 1
    # Create a pixmap for each color
    splited = [(pixmap == i) * i for i in range(1, nb_colors)]
    # Filter out empty images
    return [x for x in splited if np.any(x)]

# np.array -> [np.array]
def cropToContent_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    # Op argwhere will give us the coordinates of every non-zero point
    true_points = np.argwhere(pixmap)
    if len(true_points) == 0:
        return []
    # Take the smallest points and use them as the top left of our crop
    top_left = true_points.min(axis=0)
    # Take the largest points and use them as the bottom right of our crop
    bottom_right = true_points.max(axis=0)
    # Crop inside the defined rectangle
    pixmap = pixmap[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    return [pixmap]

# np.array -> [np.array]
def splitH_unlifted(pixmap):
    """ Split horizontally an image """
    h = pixmap.shape[0]
    if h % 2 == 1:
        h = h // 2
        return [pixmap[:h,:], pixmap[h+1:,:]]
    else:
        h = h // 2
        return [pixmap[:h,:], pixmap[h:,:]]

# np.array -> [np.array]
def negative_unlifted(pixmap):
    """ Compute the negative of an image (and conserve the color) """
    negative = np.logical_not(pixmap).astype(int)
    color = max(pixmap.max(), 1)
    return [negative * color]

# np.array -> [np.array]
def rotation_unlifted(pixmap):
    """ Compute an image with rotation"""
    #90度反時計回り
    ans1 = np.rot90(np.array(pixmap))
    #180度反時計回り
    ans2 = np.rot90(np.array(pixmap), 2) 
    #270度反時計回り
    ans3 = np.rot90(np.array(pixmap), 3)
    return [ans1, ans2, ans3]
    
def unrotation_unlifted(pixmap):
    #90度時計回り
    ans4 = np.rot90(np.array(pixmap),-1)
    #180度時計回り
    ans5 = np.rot90(np.array(pixmap), -2)
    #270度時計回り
    ans6 = np.rot90(np.array(pixmap), -3) 
    return [ans4, ans5, ans6]

# np.array -> [np.array]
def flip_unlifted(pixmap):
    """ Compute an image with flip"""
    #左右反転
    ans1 = np.fliplr(np.array(pixmap))
    #上下反転
    ans2 = np.flipud(np.array(pixmap))
    #転置
    ans3 = np.transpose(np.array(pixmap))
    return [ans1, ans2, ans3]    

# np.array -> [np.array]
def splitV_unlifted(pixmap):
    """ Split horizontally an image """
    v = pixmap.shape[1]
    if v % 2 == 1:
        v = v // 2
        return [pixmap[:,:v], pixmap[:,v+1:]]
    else:
        v = v // 2
        return [pixmap[:,:v], pixmap[:,v:]]
    
def change_from0_unlifted(pixmap):
    """ change color from 0 to any """
    ans = []
    for i in range(10):
        if i == 0:
            continue
        res = pixmap.copy()
        res[pixmap==0] = i
        ans.append(res)
    return ans

def change_from1_unlifted(pixmap):
    """ change color from 1 to any """
    ans = []
    for i in range(10):
        if i == 1:
            continue
        res = pixmap.copy()
        res[pixmap==1] = i
        ans.append(res)
    return ans

def change_from2_unlifted(pixmap):
    """ change color from 2 to any """
    ans = []
    for i in range(10):
        if i == 2:
            continue
        res = pixmap.copy()
        res[pixmap==2] = i
        ans.append(res)
    return ans

def change_from3_unlifted(pixmap):
    """ change color from 3 to any """
    ans = []
    for i in range(10):
        if i == 3:
            continue
        res = pixmap.copy()
        res[pixmap==3] = i
        ans.append(res)
    return ans

def change_from4_unlifted(pixmap):
    """ change color from 4 to any """
    ans = []
    for i in range(10):
        if i == 4:
            continue
        res = pixmap.copy()
        res[pixmap==4] = i
        ans.append(res)
    return ans

def change_from5_unlifted(pixmap):
    """ change color from 5 to any """
    ans = []
    for i in range(10):
        if i == 5:
            continue
        res = pixmap.copy()
        res[pixmap==5] = i
        ans.append(res)
    return ans

def change_from6_unlifted(pixmap):
    """ change color from 6 to any """
    ans = []
    for i in range(10):
        if i == 6:
            continue
        res = pixmap.copy()
        res[pixmap==6] = i
        ans.append(res)
    return ans

def change_from7_unlifted(pixmap):
    """ change color from 7 to any """
    ans = []
    for i in range(10):
        if i == 7:
            continue
        res = pixmap.copy()
        res[pixmap==7] = i
        ans.append(res)
    return ans

def change_from8_unlifted(pixmap):
    """ change color from 8 to any """
    ans = []
    for i in range(10):
        if i == 8:
            continue
        res = pixmap.copy()
        res[pixmap==8] = i
        ans.append(res)
    return ans

def change_from9_unlifted(pixmap):
    """ change color from 9 to any """
    ans = []
    for i in range(10):
        if i == 9:
            continue
        res = pixmap.copy()
        res[pixmap==9] = i
        ans.append(res)
    return ans

def copy_horizontal_unlifted(pixmap):
    """ copy picture and add horizontally """
    return [pixmap.repeat(2, axis=0)]

def copy_vertical_unlifted(pixmap):
    """ copy picture and add vertically """
    return [pixmap.repeat(2, axis=1)]

def crop_inside_unlifted(pixmap):
    """ Crop inside an image to fit exactly the non 0 pixels """
    # Op argwhere will give us the coordinates of every non-zero point
    true_points = np.argwhere(pixmap)
    if len(true_points) == 0:
        return []
    # Take the smallest points and use them as the top left of our crop
    top_left = true_points.min(axis=0)
    # Take the largest points and use them as the bottom right of our crop
    bottom_right = true_points.max(axis=0)
    # Crop inside the defined rectangle
    pixmap = pixmap[top_left[0]+1:bottom_right[0], top_left[1]+1:bottom_right[1]]
    return [pixmap]

def edge_red_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["red"]
    res[:,0] = color2num["red"]
    res[H-1,:] = color2num["red"]
    res[:,W-1] = color2num["red"]
    return [res]

def edge_blue_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["blue"]
    res[:,0] = color2num["blue"]
    res[H-1,:] = color2num["blue"]
    res[:,W-1] = color2num["blue"]
    return [res]

def edge_green_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["green"]
    res[:,0] = color2num["green"]
    res[H-1,:] = color2num["green"]
    res[:,W-1] = color2num["green"]
    return [res]

def edge_yellow_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["yellow"]
    res[:,0] = color2num["yellow"]
    res[H-1,:] = color2num["yellow"]
    res[:,W-1] = color2num["yellow"]
    return [res]

def edge_gray_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["gray"]
    res[:,0] = color2num["gray"]
    res[H-1,:] = color2num["gray"]
    res[:,W-1] = color2num["gray"]
    return [res]
    
def edge_magenta_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["magenta"]
    res[:,0] = color2num["magenta"]
    res[H-1,:] = color2num["magenta"]
    res[:,W-1] = color2num["magenta"]
    return [res]

def edge_orange_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["orange"]
    res[:,0] = color2num["orange"]
    res[H-1,:] = color2num["orange"]
    res[:,W-1] = color2num["orange"]
    return [res]

def edge_sky_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["sky"]
    res[:,0] = color2num["sky"]
    res[H-1,:] = color2num["sky"]
    res[:,W-1] = color2num["sky"]
    return [res]

def edge_brown_colorize_unlifted(pixmap):
    """ colorize edges in the picture """
    H, W = pixmap.shape
    res = pixmap.copy()
    res[0,:] = color2num["brown"]
    res[:,0] = color2num["brown"]
    res[H-1,:] = color2num["brown"]
    res[:,W-1] = color2num["brown"]
    return [res]

def expand_twice_unlifted(arr):
    return [arr.repeat(2, axis=1).repeat(2,axis=0)]

def expand_three_unlifted(arr):
    return [arr.repeat(3, axis=1).repeat(3,axis=0)]

def expand_four_unlifted(arr):
    return [arr.repeat(4, axis=1).repeat(4,axis=0)]

def horizontal_combi_twice_unlifted(arr):
    return [np.concatenate((arr,arr),axis=1)]

def horizontal_combi_three_unlifted(arr):
    return [np.concatenate((arr,arr,arr),axis=1)]

def horizontal_combi_four_unlifted(arr):
    return [np.concatenate((arr,arr,arr,arr),axis=1)]

def vertical_combi_twice_unlifted(arr):
    return [np.concatenate((arr,arr),axis=0)]

def vertical_combi_three_unlifted(arr):
    return [np.concatenate((arr,arr,arr),axis=0)]

def vertical_combi_four_unlifted(arr):
    return [np.concatenate((arr,arr,arr,arr),axis=0)]

def closed_area_to_yellow_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["yellow"]
    return [y]

def closed_area_to_blue_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["blue"]
    return [y]

def closed_area_to_red_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["red"]
    return [y]

def closed_area_to_green_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["green"]
    return [y]

def closed_area_to_orange_unlifted(arr):
    # depth first search
    H, W = arr.shape
    Dy = [0, -1, 0, 1]
    Dx = [1, 0, -1, 0]
    arr_padded = np.pad(arr, ((1,1),(1,1)), "constant", constant_values=0)
    searched = np.zeros(arr_padded.shape, dtype=bool)
    searched[0, 0] = True
    q = [(0, 0)]
    while q:
        y, x = q.pop()
        for dy, dx in zip(Dy, Dx):
            y_, x_ = y+dy, x+dx
            if not 0 <= y_ < H+2 or not 0 <= x_ < W+2:
                continue
            if not searched[y_][x_] and arr_padded[y_][x_]==0:
                q.append((y_, x_))
                searched[y_, x_] = True
    res = searched[1:-1, 1:-1]
    res |= arr!= color2num["black"]
    y = arr.copy()
    y[~res] = color2num["orange"]
    return [y]

def delete_last_row_unlifted(arr):
    return [arr[:-1,:]]

def delete_last_column_unlifted(arr):
    return [arr[:,:-1]]

def delete_last2_row_unlifted(arr):
    return [arr[:-2,:]]

def delete_last2_column_unlifted(arr):
    return [arr[:,:-2]]

def delete_last3_row_unlifted(arr):
    return [arr[:-3,:]]

def delete_last3_column_unlifted(arr):
    return [arr[:,:-3]]

def kronecker_expansion_unlifted(arr):
    flg = arr > 0
    return [np.kron(flg,arr)]

def expansion_fliplr_unlifted(pixmap):
    """ Expand an image by connecting flipped image"""
    tmp = np.fliplr(np.array(pixmap))
    ans1 = np.concatenate((pixmap, tmp), axis=1)
    ans2 = np.concatenate((tmp, pixmap), axis=1)
    return [ans1, ans2]

def expansion_flipud_unlifted(pixmap):
    """ Expand an image by connecting flipped image"""
    tmp = np.flipud(np.array(pixmap))
    ans1 = np.concatenate((pixmap, tmp), axis=0)
    ans2 = np.concatenate((tmp, pixmap), axis=0)
    return [ans1, ans2]

# np.array -> [np.array]
def overlapV_unlifted(pixmap):
    """ Split vertically an image and overlap"""
    v = pixmap.shape[1]
    if v % 2 == 1:
        v = v // 2
        a, b = pixmap[:,:v], pixmap[:,v+1:]
    else:
        v = v // 2
        a, b = pixmap[:,:v], pixmap[:,v:]
    flg_a = a > 0
    flg_b = b > 0
    return [np.logical_and(flg_a, flg_b).astype("int32"), np.logical_or(flg_a, flg_b).astype("int32"), np.logical_xor(flg_a, flg_b).astype("int32")]

# np.array -> [np.array]
def overlapH_unlifted(pixmap):
    """ Split horizontally an image and overlap"""
    h = pixmap.shape[0]
    if h % 2 == 1:
        h = h // 2
        a, b = [pixmap[:h,:], pixmap[h+1:,:]]
    else:
        h = h // 2
        a, b = [pixmap[:h,:], pixmap[h:,:]]
    flg_a = a > 0
    flg_b = b > 0
    return [np.logical_and(flg_a, flg_b).astype("int32"), np.logical_or(flg_a, flg_b).astype("int32"), np.logical_xor(flg_a, flg_b).astype("int32")]

def period_length_copy_horizontal_unlifted(pixmap):
    """ copy part of the picure and add horizontally"""
    H, W = pixmap.shape
    period = 1
    while True:
        cycled = np.pad(pixmap[:period, :], ((0,H-period),(0,0)), 'wrap')
        if (cycled==pixmap).all():
            break
        period += 1
            
    y = pixmap[:period, :]  # clop one period
    y = np.pad(y, ((0,2*H-period),(0,0)), 'wrap')  # cycle
    return [y]

def period_length_copy_vertical_unlifted(pixmap):
    """ copy part of the picure and add horizontally"""
    H, W = pixmap.shape
    period = 1
    while True:
        cycled = np.pad(pixmap[:,:period], ((0,0),(W-period,0)), 'wrap')
        if (cycled==pixmap).all():
            break
        period += 1
            
    y = pixmap[:,:period]  # clop one period
    y = np.pad(y, ((0,0),(2*W-period,0)), 'wrap')  # cycle
    return [y]

def three_diagonal_pattern_unlifted(x):
    H, W = x.shape
    colors = [0, 0, 0]
    for yy in range(H):
        for xx in range(W):
            color = x[yy, xx]
            if color != 0:
                colors[(yy+xx)%3] = color
    y = x.copy()
    for yy in range(H):
        for xx in range(W):
            y[yy, xx] = colors[(yy+xx)%3]
    return [y]

def down_the_red_line_unlifted(x):
    y = x.copy()
    red = color2num["red"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==red)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = red
    return [y]

def down_the_blue_line_unlifted(x):
    y = x.copy()
    blue = color2num["blue"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==blue)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = blue
    return [y]

def down_the_green_line_unlifted(x):
    y = x.copy()
    green = color2num["green"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==green)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = green
    return [y]

def down_the_yellow_line_unlifted(x):
    y = x.copy()
    yellow = color2num["yellow"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==yellow)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = yellow
    return [y]

def down_the_orange_line_unlifted(x):
    y = x.copy()
    orange = color2num["orange"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==orange)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = orange
    return [y]

def down_the_gray_line_unlifted(x):
    y = x.copy()
    gray = color2num["gray"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==gray)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = gray
    return [y]

def down_the_sky_line_unlifted(x):
    y = x.copy()
    sky = color2num["sky"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==sky)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = sky
    return [y]

def down_the_magenta_line_unlifted(x):
    y = x.copy()
    magenta = color2num["magenta"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==magenta)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = magenta
    return [y]

def down_the_brown_line_unlifted(x):
    y = x.copy()
    brown = color2num["brown"]
    idx_set = set(tuple(idx) for idx in np.array(np.where(x==brown)).T)
    for i in idx_set:
        y[i[0]:,i[1]] = brown
    return [y]

def sky_move_unlifted(x):
    H, W = x.shape
    color = color2num["sky"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def red_move_unlifted(x):
    H, W = x.shape
    color = color2num["red"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x]   
    
def blue_move_unlifted(x):
    H, W = x.shape
    color = color2num["blue"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def green_move_unlifted(x):
    H, W = x.shape
    color = color2num["green"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def yellow_move_unlifted(x):
    H, W = x.shape
    color = color2num["yellow"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def magenta_move_unlifted(x):
    H, W = x.shape
    color = color2num["magenta"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def brown_move_unlifted(x):
    H, W = x.shape
    color = color2num["brown"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x] 
    
def gray_move_unlifted(x):
    H, W = x.shape
    color = color2num["gray"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    ans = []
    if len(idx_list) >0:
        y1 = x.copy()
        y2 = x.copy()
        y3 = x.copy()
        y4 = x.copy()
        for idx in idx_list:
            y1[idx] = color2num["black"]
            y2[idx] = color2num["black"]
            y3[idx] = color2num["black"]
            y4[idx] = color2num["black"]
        obj_idx1 = set((idx[0]+1, idx[1]) for idx in idx_list if 0<=idx[0]+1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx2 = set((idx[0]-1, idx[1]) for idx in idx_list if 0<=idx[0]-1<=H-1 and 0<= idx[1] <=W-1)
        obj_idx3 = set((idx[0], idx[1]+1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]+1 <=W-1)
        obj_idx4 = set((idx[0], idx[1]-1) for idx in idx_list if 0<=idx[0]<=H-1 and 0<= idx[1]-1 <=W-1)
        for idx in obj_idx1:
            y1[idx] = color
        for idx in obj_idx2:
            y2[idx] = color
        for idx in obj_idx3:
            y3[idx] = color
        for idx in obj_idx4:
            y4[idx] = color
        return [y1, y2, y3, y4]
    else:
        return [x]  
    
def delete_blue_unlifted(x):
    color = color2num["blue"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_red_unlifted(x):
    color = color2num["red"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_yellow_unlifted(x):
    color = color2num["yellow"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_green_unlifted(x):
    color = color2num["green"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_gray_unlifted(x):
    color = color2num["gray"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_magenta_unlifted(x):
    color = color2num["magenta"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_sky_unlifted(x):
    color = color2num["sky"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def delete_brown_unlifted(x):
    color = color2num["brown"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y1 = x.copy()
    if len(idx_list) >0:
        for idx in idx_list:
            y1[idx] = color2num["black"]
    return [y1]

def horizontal_blue_unlifted(x):
    color = color2num["blue"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_red_unlifted(x):
    color = color2num["red"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_green_unlifted(x):
    color = color2num["green"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_yellow_unlifted(x):
    color = color2num["yellow"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_magenta_unlifted(x):
    color = color2num["magenta"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_brown_unlifted(x):
    color = color2num["brown"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_magenta_unlifted(x):
    color = color2num["gray"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_sky_unlifted(x):
    color = color2num["sky"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def horizontal_gray_unlifted(x):
    color = color2num["gray"]
    idx_list = [tuple(idx) for idx in np.array(np.where(x==color)).T]
    y = x.copy()
    if len(idx_list) == 1:
        y[:,idx_list[0][1]] = color
    return [y]

def expand_by_unique_colors_unlifted(pixmap):
    """ copy picture and add horizontally """
    x = pixmap.copy()
    unique_num = len(np.unique(x[x > 0]))
    return [x.repeat(unique_num, axis=0).repeat(unique_num, axis=1)]

We now write functions that take a list of images and transform it to a new list.

In [6]:
# [np.array] -> [np.array]
def identity(x: [np.array]):
    return x

# [np.array] -> [np.array]
def tail(x):
    if len(x) > 1:
        return x[1:]
    else:
        return x

# [np.array] -> [np.array]
def init(x):
    if len(x) > 1:
        return x[:1]
    else:
        return x

# [np.array] -> [np.array]
def union(x):
    """ Compute the pixel union of all images in the list. """
    if len(x) < 2:
        return x
    
    # Make sure everybody have the same shape
    first_shape = tuple(x[0].shape)
    for pixmap in x[1:]:
        if first_shape != tuple(pixmap.shape):
            return []
    
    return [np.bitwise_or.reduce(np.array(x).astype(int))]
    
def intersect(x):
    """ Compute the pixel intersection of all images in the list. """
    if len(x) < 2:
        return x
    
    # Make sure everybody have the same shape
    first_shape = tuple(x[0].shape)
    for pixmap in x[1:]:
        if first_shape != tuple(pixmap.shape):
            return []
    
    return [(np.prod(np.array(x), axis=0) > 0).astype(int)]

def sortByColor(xs):
    """ Sort pictures by increasing color id. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: x.max()))

def sortByWeight(xs):
    """ Sort images by how many non zero pixels are contained. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: (x>0).sum()))

def reverse(x):
    """ Reverse the order of a list of images. """
    return x[::-1]

## Composition of functions

It is important to make sure we can chain both functions. To compose two functions `f` and `g` of type `[np.array] -> [np.array]` ; We symply call `g(f([input_image]))`.


But for each function of the type `np.array -> [np.array]` some work is required. We need to generated a *lifted version* version of them. A function `f: np.array -> [np.array]` can be turned into a function of type `[np.array] -> [np.array]` by applying `f` on each image of the input list and concatenating the results.

---
If you want to know more about the `lift` concept, have a look to the concept of [*monades*](https://en.wikipedia.org/wiki/Monad_%28functional_programming%29). We are indeed using the *list monade*.

In [7]:
def lift(fct):
    # Lift the function
    def lifted_function(xs):
        list_of_results = [fct(x) for x in xs]
        return list(itertools.chain(*list_of_results))
    # Give a nice name to the lifted function
    import re
    lifted_function.__name__ = re.sub('_unlifted$', '_lifted', fct.__name__)
    return lifted_function

cropToContent = lift(cropToContent_unlifted)
groupByColor = lift(groupByColor_unlifted)
splitH = lift(splitH_unlifted)
negative = lift(negative_unlifted)
rotation = lift(rotation_unlifted)
unrotation = lift(unrotation_unlifted)
flip = lift(flip_unlifted)
splitV = lift(splitV_unlifted)
change_from0 = lift(change_from0_unlifted)
change_from1 = lift(change_from1_unlifted)
change_from2 = lift(change_from2_unlifted)
change_from3 = lift(change_from3_unlifted)
change_from4 = lift(change_from4_unlifted)
change_from5 = lift(change_from5_unlifted)
change_from6 = lift(change_from6_unlifted)
change_from7 = lift(change_from7_unlifted)
change_from8 = lift(change_from8_unlifted)
change_from9 = lift(change_from9_unlifted)
copy_horizontal = lift(copy_horizontal_unlifted)
copy_vertical = lift(copy_vertical_unlifted)
crop_inside = lift(crop_inside_unlifted)
edge_red_colorize = lift(edge_red_colorize_unlifted)
edge_blue_colorize = lift(edge_blue_colorize_unlifted)
edge_green_colorize = lift(edge_green_colorize_unlifted)
edge_yellow_colorize = lift(edge_yellow_colorize_unlifted)
edge_gray_colorize = lift(edge_gray_colorize_unlifted)
edge_magenta_colorize = lift(edge_magenta_colorize_unlifted)
edge_orange_colorize = lift(edge_orange_colorize_unlifted)
edge_sky_colorize = lift(edge_sky_colorize_unlifted)
edge_brown_colorize = lift(edge_brown_colorize_unlifted)
expand_twice = lift(expand_twice_unlifted)
expand_three = lift(expand_three_unlifted)
expand_four = lift(expand_four_unlifted)
horizontal_combi_twice = lift(horizontal_combi_twice_unlifted)
horizontal_three_twice = lift(horizontal_combi_three_unlifted)
horizontal_four_twice = lift(horizontal_combi_four_unlifted)
vertical_combi_twice = lift(vertical_combi_twice_unlifted)
vertical_combi_three = lift(vertical_combi_three_unlifted)
vertical_combi_four = lift(vertical_combi_four_unlifted)
closed_area_to_yellow = lift(closed_area_to_yellow_unlifted)
closed_area_to_orange = lift(closed_area_to_orange_unlifted)
closed_area_to_green = lift(closed_area_to_green_unlifted)
closed_area_to_red = lift(closed_area_to_red_unlifted)
closed_area_to_blue = lift(closed_area_to_blue_unlifted)
delete_last_row = lift(delete_last_row_unlifted)
delete_last_column = lift(delete_last_column_unlifted)
delete_last2_row = lift(delete_last2_row_unlifted)
delete_last2_column = lift(delete_last2_column_unlifted)
delete_last3_row = lift(delete_last3_row_unlifted)
delete_last3_column = lift(delete_last3_column_unlifted)
kronecker_expansion = lift(kronecker_expansion_unlifted)
expansion_fliplr = lift(expansion_fliplr_unlifted)
expansion_flipud = lift(expansion_flipud_unlifted)
overlapV = lift(overlapV_unlifted)
overlapH = lift(overlapH_unlifted)
period_length_copy_horizontal = lift(period_length_copy_horizontal_unlifted)
period_length_copy_vertical = lift(period_length_copy_vertical_unlifted)
three_diagonal_pattern = lift(three_diagonal_pattern_unlifted)
down_the_red_line = lift(down_the_red_line_unlifted)
down_the_blue_line = lift(down_the_blue_line_unlifted)
down_the_yellow_line = lift(down_the_yellow_line_unlifted)
down_the_gray_line = lift(down_the_gray_line_unlifted)
down_the_green_line = lift(down_the_green_line_unlifted)
down_the_sky_line = lift(down_the_sky_line_unlifted)
down_the_magenta_line = lift(down_the_magenta_line_unlifted)
down_the_brown_line = lift(down_the_brown_line_unlifted)
down_the_orange_line = lift(down_the_orange_line_unlifted)
gray_move = lift(gray_move_unlifted)
red_move = lift(red_move_unlifted)
blue_move = lift(blue_move_unlifted)
green_move = lift(green_move_unlifted)
yellow_move = lift(yellow_move_unlifted)
brown_move = lift(brown_move_unlifted)
magenta_move = lift(magenta_move_unlifted)
sky_move = lift(sky_move_unlifted)
delete_red = lift(delete_red_unlifted)
delete_blue = lift(delete_blue_unlifted)
delete_green = lift(delete_green_unlifted)
delete_sky = lift(delete_sky_unlifted)
delete_yellow = lift(delete_yellow_unlifted)
delete_sky = lift(delete_sky_unlifted)
delete_magenta = lift(delete_magenta_unlifted)
delete_gray = lift(delete_gray_unlifted)
delete_brown = lift(delete_brown_unlifted)
horizontal_blue = lift(horizontal_blue_unlifted)
horizontal_red = lift(horizontal_red_unlifted)
horizontal_yellow = lift(horizontal_yellow_unlifted)
horizontal_brown = lift(horizontal_brown_unlifted)
horizontal_green = lift(horizontal_green_unlifted)
horizontal_magenta = lift(horizontal_magenta_unlifted)
horizontal_sky = lift(horizontal_sky_unlifted)
horizontal_gray = lift(horizontal_gray_unlifted)
expand_by_unique_colors = lift(expand_by_unique_colors_unlifted)

# Program evaluation


We define our building blocks for programs (the functions in our DSL). We will define a program as a list of functions from our DSL ; `program: [[np.array] -> [np.array]]`. The instructions in our programs will be executed *from left to right*. This mean that if we want to first `splitByColor` and then compute the `negative` of the image, we need to write `[splitByColor, negative]` in this order.

In [8]:
def program_desc(program):
    """ Create a human readable description of a program. """
    desc = [x.__name__ for x in program]
    return(' >> '.join(desc))

# Display the program description alongside its output
program = [splitH, groupByColor, negative, intersect]
print(program_desc(program))

splitH_lifted >> groupByColor_lifted >> negative_lifted >> intersect


## The evaluation method
We want to run and evaluate a such program on a pictures and then recover the result. This logic is realised by the `evaluate` function.

In [9]:
def evaluate(program: [], input_image: np.array):
    # Make sure the input is a np.array
    input_image = np.array(input_image)
    assert type(input_image) == np.ndarray
    
    # Apply each function on the image
    image_list = [input_image]
    for fct in program:
        # Apply the function
        image_list = fct(image_list)
        # Filter out empty images
        image_list = [img for img in image_list if img.shape[0] > 0 and img.shape[1] > 0]
        # Break if there is no data
        if image_list == []:
            return []
    return image_list        

# Program generation (Genetic Algorithm)

We now have a simple and powerful language to express various transformation on images. But someone or something still have to write the actual program that can solve a task. In this part, we will implement a naive but somewhat efficient genetic algorithm that will be able to find by itself the solution to a task.

The strategy will be as follow:

* We generate random program with one node, and then run them. We keep the best solution (the *elites* of our population).
* Starting from this best solutions, we create new program though mutation. We avaluate them again and update our collection of elite.
* We continue doing this process again and again... until a solution is found.

---

Since we use multiple fitness function, our aproache can be qualified of [multi-objectives](https://en.wikipedia.org/wiki/Multi-objective_optimization) : we try to optimise multiple objectives at the same time.

Our *elites* can be understood as an approximation of the pareto surface (collection of pareto optimal solution). In our specific case, when a solution to the task exists in our DSL, their exists a global minimum that will be smaller than any candidate. In a such case the pareto surface is reduced to a single point. Nethertheless, this is a good image to keep in mind to understand what the collection of *elites* represent.

## Is a program solution ?

In [10]:
def are_two_images_equals(a, b):
    if tuple(a.shape) == tuple(b.shape):
        if (np.abs(b-a) < 1).all():
            return True
    return False

def is_solution(program, task, verbose=True):
    for sample in task: # For each pair input/output
        i = np.array(sample['input'])
        o = np.array(sample['output'])

        # Evaluate the program on the input
        images = evaluate(program, i)
        if len(images) < 1:
            return False
        
        # The solution should be in the 3 first outputs ???
        images = images[:10]
        
        # Check if the output is in the 3 images produced
        is_program_of_for_sample = any([are_two_images_equals(x, o) for x in images])
        if not is_program_of_for_sample:
            return False
    
    return True

## Fitness

To help our algorithm progress in the right direction, we need a way to give a score to an existing program. The smaller is the score of the program, the closer we are to the solution. One can think of this score as a distance of our program to the optimal solution.

Notice that one can think of this program as a minimization problem (minimize `score`) or maximization problem (minimize `-score`). On machine learning it is common to minimise a distance wereas in genetic algorithm literature you can read that we maximize the fitness of an agent^1. Both convention work perfectly, but it is more convenient if we choose one and stick to it. Therefore, we will MINIMIZE the score of our programs.

Because we can't really comme up with one single good score function that would describe well the progression of the algorithm on all task of the dataset, we will evaluate how our program perform on different aspects through a collection of them.

^1: The reason you see maximization and positive score in Genetic Programming literature is that you need all your values to be positive in order to build a probability distribution over your population. Since we use an elitist algorithm instead of a sampling of the population for reproduction, we do not need this restriction.

In [11]:
def width_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right width. Less is better."""
    return np.abs(predicted.shape[0] - expected_output.shape[0])

def height_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right height. Less is better."""
    return np.abs(predicted.shape[1] - expected_output.shape[1])

def activated_pixels_fitness(p, e):
    """ How close the predicted image to have the right pixels. Less is better."""
    shape = (max(p.shape[0], e.shape[0]), max(p.shape[1], e.shape[1]))
    diff = np.zeros(shape, dtype=int)
    diff[0:p.shape[0], 0:p.shape[1]] = (p > 0).astype(int)
    diff[0:e.shape[0], 0:e.shape[1]] -= (e > 0).astype(int)
    
    return (diff != 0).sum()

def colors_fitness(p, e):
    p_colors = np.unique(p)
    e_colors = np.unique(e)
    
    nb_inter = len(np.intersect1d(p_colors, e_colors))

    return (len(p_colors) - nb_inter) + (len(e_colors) - nb_inter)

def colors_num_sum_fitness(p, e):
    p_h, p_w = p.shape
    e_h, e_w = e.shape
    if p_h == e_h and p_w == e_w:
        p_colors_sum = np.sum(np.sum(p))
        e_colors_sum = np.sum(np.sum(e))
        return np.abs(p_colors_sum - e_colors_sum)
    else:
        return 10000 # big penalty

fitness_functions = [height_fitness, width_fitness,  colors_fitness, activated_pixels_fitness, colors_num_sum_fitness]
coefficients = [1, 1, 1, 1, 1]

In [12]:
def product_less(a, b):
    """ Return True iff the two tuples a and b respect a<b for the partial order. """
    a = np.array(a)
    b = np.array(b)
    return (np.array(a) < np.array(b)).all()

We now write a function that evaluate the fitness of a program on a task.

In [13]:
# ([[np.array] -> [np.array]], Taks) -> (int, int, ..., int)
def evaluate_fitness(program, task, coefficients):
    """ Take a program and a task, and return its fitness score as a tuple. """
    score = np.zeros((len(fitness_functions)))
    
    # For each sample
    for sample in task:
        i = np.array(sample['input'])
        o = np.array(sample['output'])
        
        # For each fitness function
        for index, fitness_function in enumerate(fitness_functions):
            images = evaluate(program, i)
            if images == []: # Penalize no prediction!
                score[index] += 500
            else: # Take only the score of the first output
                score[index] = fitness_function(images[0], o)
    total_score = sum([score[di] * coefficients[di] for di in range(len(score))])
    return total_score #tuple(score)

## Asexual reproduction

Now that we can compare two programs we need a way to generate some of them. We will generate them randomly from a pool of best candidate.
For the initial run, and also to be able to evaluate fresh candidates, we will also allow spontaneous generation of new born one instruction programs.

In [14]:
def build_candidates(allowed_nodes=[identity], best_candidates=[], nb_candidates=300):
    """
    Create a poll of fresh candidates using the `allowed_nodes`.
    The pool contain a mix of new single instructions programs
    and mutations of the best candidates.
    """
    new_candidates = []
    length_limit = 4 # Maximal length of a program
    
    def random_node():
        return random.choice(allowed_nodes)
    
    # Until we have enougth new candidates
    while(len(new_candidates) < nb_candidates):
        # Add 10 new programs
        for i in range(5):
            new_candidates += [[random_node()]]
        
        # Create new programs based on each best candidate
        for best_program in best_candidates:
            # Add one op on its right but limit the length of the program
            if len(best_program) < length_limit - 1:
                new_candidates += [[random_node()] + best_program]
            # Add one op on its left but limit the length of the program
            if len(best_program) < length_limit - 1:
                new_candidates += [best_program + [random_node()]]
            # Mutate one instruction of the existing program
            new_candidates += [list(best_program)]
            new_candidates[-1][random.randrange(0, len(best_program))] = random_node()
   
    # Truncate if we have too many candidates
    np.random.shuffle(new_candidates)
    return new_candidates[:nb_candidates]

# Test the function by building some candidates
len(build_candidates(allowed_nodes=[identity], best_candidates=[[identity]], nb_candidates=3))

3

## Find a program given a task

This is the last step to our genetic algorithm. We have all the building blocks:
 * Generating both new programs and mutation of existing solutions
 * Evaluating the fitness score of a program
 * Comparing two programs to know if one perform better than the other
 * Detecting when a solution was found
 
We can now write a function that will keep generating programs with increasing complexity until a solution is found.

Using our partial order, we are going to keep the best candidates. Because the order is partial,
there is no bound on how many uncomparables candidates we may have at a given iteration.

In [15]:
def build_model(task, coefficients, max_iterations=20, verbose=True):
    candidates_nodes = [
        tail, init, union, intersect,
        sortByColor, sortByWeight, reverse,
        cropToContent, groupByColor, splitH,
        negative, rotation, unrotation, flip, splitV,
        change_from0, change_from1, change_from2, change_from3, change_from4,
        change_from5, change_from6, change_from7, change_from8, change_from9,
        copy_horizontal, copy_vertical, crop_inside,
        edge_red_colorize, edge_blue_colorize, edge_green_colorize,
        edge_yellow_colorize, edge_gray_colorize, edge_magenta_colorize,
        edge_orange_colorize, edge_sky_colorize, edge_brown_colorize,
        expand_twice, expand_three, expand_four,
        horizontal_combi_twice, horizontal_three_twice, horizontal_four_twice,
        vertical_combi_twice, vertical_combi_three, vertical_combi_four, closed_area_to_yellow,
        closed_area_to_blue, closed_area_to_red, closed_area_to_orange, closed_area_to_green, 
        delete_last_row, delete_last_column, delete_last2_row, delete_last2_column, 
        delete_last3_row, delete_last3_column, kronecker_expansion, 
        expansion_fliplr, expansion_flipud,overlapV, overlapH,
        period_length_copy_horizontal, period_length_copy_vertical, three_diagonal_pattern,
        down_the_red_line, down_the_blue_line, down_the_yellow_line, down_the_gray_line, 
        down_the_green_line, down_the_sky_line, down_the_magenta_line, down_the_brown_line, down_the_orange_line,
        sky_move, gray_move, red_move, blue_move, green_move, yellow_move, brown_move, magenta_move, 
        delete_sky, delete_gray, delete_red, delete_blue, delete_green, delete_yellow, delete_brown, delete_magenta, 
        horizontal_sky, horizontal_gray, horizontal_red, horizontal_blue, horizontal_green, horizontal_yellow, 
        horizontal_brown, horizontal_magenta, expand_by_unique_colors
    ]
    
    #if verbose:
    #    print("Candidates nodes are:", [program_desc([n]) for n in candidates_nodes])
    #    print()

    best_candidates = {} # A dictionary of {score:candidate}
    for i in range(max_iterations):
        if verbose:
            print("Iteration ", i+1)
            print("-" * 10)
        
        # Create a list of candidates
        candidates = build_candidates(candidates_nodes, best_candidates.values())
        
        # Keep candidates with best fitness.
        # They will be stored in the `best_candidates` dictionary
        # where the key of each program is its fitness score.
        for candidate in candidates:
            score = evaluate_fitness(candidate, task, coefficients)
            is_uncomparable = True # True if we cannot compare the two candidate's scores
            
            # Compare the new candidate to the existing best candidates
            best_candidates_items = list(best_candidates.items())
            for best_score, best_candidate in best_candidates_items:
                if product_less(score, best_score):
                    # Remove previous best candidate and add the new one
                    del best_candidates[best_score]
                    best_candidates[score] = candidate
                    is_uncomparable = False # The candidates are comparable
                if product_less(best_score, score) or best_score == score:
                    is_uncomparable = False # The candidates are comparable
            if is_uncomparable: # The two candidates are uncomparable
                best_candidates[score] = candidate

        # For each best candidate, we look if we have an answer
        for program in best_candidates.values():
            if is_solution(program, task):
                return program
            
        # Give some informations by selecting a random candidate
        if verbose:
            print("Best candidates length:", len(best_candidates))
            random_candidate_score = random.choice(list(best_candidates.keys()))
            print("Random candidate score:", random_candidate_score)
            print("Random candidate implementation:", program_desc(best_candidates[random_candidate_score]))
    return None

# Solve the task

In [16]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

sample_sub = pd.read_csv(data_path/'sample_submission.csv')
sample_sub = sample_sub.set_index('output_id')

In [17]:
mode = "test"
if mode=='eval':
    task_path = evaluation_path
elif mode=='train':
    task_path = training_path
elif mode=='test':
    task_path = test_path
    
all_task_ids = sorted(os.listdir(task_path))
for i in range(len(all_task_ids)):
    task_file = str(task_path / all_task_ids[i])
    with open(task_file, 'r') as f:
        task = json.load(f)
    program = build_model(task['train'], coefficients, verbose=False)
    if program is None:
        print(str(i) +", No program was found in "+str(task_file))
    else:
        print(task_file)
        print(str(i) +", Found program:", program_desc(program))
        # produce output for the test input
        if mode == 'test':
            for task_num in range(len(task["test"])):
                images = evaluate(program, np.array(task['test'][task_num]["input"]))
                images = images[:3] # The solution should be in the 3 first outputs
                preds_list = [flattener(image.astype(int).tolist()) for image in images]
                sample_sub.loc[f'{all_task_ids[i][:-5]}_{task_num}','output'] = ' '.join(preds_list)

0, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/00576224.json
1, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/009d5c81.json
2, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/00dbd492.json
3, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/03560426.json
4, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/05a7bcf2.json
5, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/0607ce86.json
6, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/0692e18c.json
7, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/070dd51e.json
8, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/08573cc6.json
9, No program was found in /kaggle/input/abstraction-and-reasoning-challenge/test/0934a4d8.json
10, No program was found in /kaggle/inpu

In [18]:
sample_sub.to_csv('submission.csv')
sample_sub.head()

,output
output_id,
00576224_0,|32|78| |32|78| |00|00|
009d5c81_0,|00000000000000|00000888888888|00000800080808|...
00dbd492_0,|00000000000222220000|02222222220200020000|020...
03560426_0,|0000000000|0000000000|0000000000|0000000000|0...
05a7bcf2_0,|000000000020000000080000000000|00000000002220...


# Conclusion
* Extend the DSL to functions that allow solving more tasks,
* Add more fitness functions that would allow a faster convergence,
----
* Keep more than one candidate per local minima found,
* Rework the dsl as an execution graph (cf: tensorflow / onnx neural net graphs),
* Add speciation inspired from Neat / Neat-GP
* Sample the candidate pool with probabilities according to the best candidates scores,
* Add *sexual reproduction* to the programs, aka crossover.